# <u><center>Project 2 - Part 7 (Core)
- Authored By: Eric N. Valdez
- Date: 04/19/2024

In [1]:
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import json
with open('config/filepaths.json') as f:
    FPATHS = json.load(f)
st.title("Movies IMDB")

2024-04-22 02:42:01.546 
  command:

    streamlit run C:\Users\Valde\anaconda3\envs\dojo-env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

### For this project you will create a streamlit app to get predictions from your best model.

# Part 1: Preparing Best Models for Streamlit
- ### Create a new Part 7 - Preparing for Streamlit  notebook.

### `In the new notebook`
- ### Define a filpaths ditionary and save it to config/filepaths.json to include file paths for each component you will save(`review below`).
- ### Copy your best models from part 6 into the new notebook.
    - #### Update your code to define the final public-facing class labels. 

In [2]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

In [3]:
from pprint import pprint
FPATHS = dict(
    data={
        "raw": {
            "full": "Data-NLP/movie_reviews_v2.csv",  # (This is the original full dataframe we already have)
            "eda": "Data-NLP/movie_reviews_v2.csv" # We haven't saved this yet
        },
        "ml": {
            "train": "Data-NLP/training-data.joblib",  # (X_train,y_train) We haven't saved this yet
            "test": "Data-NLP/testing-data.joblib",  # (X_test,y_test) We haven't saved this yet
        },
    },
    models={
        "gru": "Models/gru/", # We haven't saved this yet
        "random_forest": "Models/random_forest/rf_reg.joblib", # We haven't saved this yet
    },
    images={
        "banner": "Images/IMDB.png", # We haven't saved this yet
    },
)
pprint(FPATHS)

{'data': {'ml': {'test': 'Data-NLP/testing-data.joblib',
                 'train': 'Data-NLP/training-data.joblib'},
          'raw': {'eda': 'Data-NLP/movie_reviews_v2.csv',
                  'full': 'Data-NLP/movie_reviews_v2.csv'}},
 'images': {'banner': 'Images/IMDB.png'},
 'models': {'gru': 'Models/gru/',
            'random_forest': 'Models/random_forest/rf_reg.joblib'}}


In [4]:
# Save the filepaths 
import os, json
os.makedirs('config/', exist_ok=True)
FPATHS_FILE = 'config/filepaths.json'
with open(FPATHS_FILE, 'w') as f:
    json.dump(FPATHS, f)

In [5]:
import os
def create_directories_from_paths(nested_dict):
    """OpenAI. (2023). ChatGPT [Large language model]. https://chat.openai.com 
    Recursively create directories for file paths in a nested dictionary.
    Parameters:
    nested_dict (dict): The nested dictionary containing file paths.
    """
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            # If the value is a dictionary, recurse into it
            create_directories_from_paths(value)
        elif isinstance(value, str):
            # If the value is a string, treat it as a file path and get the directory path
            directory_path = os.path.dirname(value)
            # If the directory path is not empty and the directory does not exist, create it
            if directory_path and not os.path.exists(directory_path):
                os.makedirs(directory_path)
                print(f"Directory created: {directory_path}")

# Use the function on your FPATHS dictionary
create_directories_from_paths(FPATHS)

In [6]:
# We can access a file using our dictionary
FPATHS['data']['raw']['full']

'Data-NLP/movie_reviews_v2.csv'

In [7]:
# We can access a file using our dictionary
FPATHS['models']['random_forest']

'Models/random_forest/rf_reg.joblib'

In [8]:
# Confirm the images is in the correct location
from IPython.display import display, Markdown
Markdown(f"<img src='{FPATHS['images']['banner']}'>")

<img src='Images/IMDB.png'>

## `Saving Your Models`
- ### For your Machine Learning Model:
    - Save your training data ([X_train, y_train])
    - Save your test data ([X_test, y_test])
    - Save your target_lookup dictionary and/or your label encoder
    - Save your best model

In [9]:
import json
with open ('config/filepaths.json') as f:
    FPATHS = json.load(f)
    
FPATHS.keys()

dict_keys(['data', 'models', 'images'])

In [10]:
FPATHS

{'data': {'raw': {'full': 'Data-NLP/movie_reviews_v2.csv',
   'eda': 'Data-NLP/movie_reviews_v2.csv'},
  'ml': {'train': 'Data-NLP/training-data.joblib',
   'test': 'Data-NLP/testing-data.joblib'}},
 'models': {'gru': 'Models/gru/',
  'random_forest': 'Models/random_forest/rf_reg.joblib'},
 'images': {'banner': 'Images/IMDB.png'}}

In [11]:
def load_data(fpath):
    df = pd.read_csv(fpath)
    df = df.set_index("movie_id")
    return df

In [12]:
# Load dataset
df = load_data(fpath = FPATHS['data']['raw']['full'])
df.head()

,original_title,review,rating
movie_id,,,
7443,Chicken Run,"A guilty pleasure for me personally, as I love...",9.0
955,Mission: Impossible II,"The first underwhelmed me, but this one straig...",3.0
955,Mission: Impossible II,After quite entertainign Mission Impossible I ...,2.0
2621,Return to Me,"Okay, I will admit I can be a bit of an easy g...",9.0
4234,Scream 3,**_Scream 3’s_ lackluster screenplay and unima...,4.0


In [13]:
def load_Xy_data(fpath):
    return joblib.load(fpath)

In [14]:
# Call function for train data
X_train, y_train = load_Xy_data(fpath=FPATHS['data']['ml']['train'])
# Call function for test data
X_test, y_test = load_Xy_data(fpath=FPATHS['data']['ml']['test'])

In [15]:
import joblib
# Load training data with FPATHS dictionary
X_train, y_train = joblib.load(FPATHS['data']['ml']['train'])
X_train

array(["Easily the worst film I have ever seen that wasn't helmed by a female director. I am usually very picky when it comes to selecting movies to watch, hence my usually high satisfaction as a cinephile. I can't believe I wasted my time on this vile piece of garbage and that this ever saw a cinematic release when there MUST be thousands...NO, millions, of better things that could have been made instead. The Lumiere brothers must be rolling in their graves at how cinema has been travestied.\r\n\r\nMy only regret other than my time wasted and the scar on my brain from having watched it is the fact that unlike Metacritic, on themoviedb.org I can't give it a zero rating. Or a negative, for that matter! =)",
       "So Creeds kid as a good job and boxes on the side for reasons... and wins.\r\n\r\nBut he doesn't want to do underground boxing, he wasn't to make it big like his dad, because he's the bestest of the bestest.\r\n\r\nAnd there is a LITTLE struggle here, a little of the heros jo

In [16]:
# Functionalize loading train or test

def load_Xy_data(fpath):
    return joblib.load(fpath)

In [17]:
X_test, y_test = load_Xy_data(fpath = FPATHS['data']['ml']['test'])
X_test.head()

movie_id
361743    > Loved this movie.  Even had me guessing to t...
447200    _Die Hard in a (X)_ is a pretty classic movie ...
8696      Another one of those movies I watched decades ...
416160    I love the idea of these flip-the-script horro...
351211    It's one of those crap remakes, where they tak...
Name: review, dtype: object

- ### For your  Deep NLP model:
    - Save your training data (train_ds)
    - Save your test data (test_ds)
    - Save your best neural network.
        - `Reminder:`use safe_format = 'tf' to save the model in a folder of repo-friendly files.

In [18]:
from pprint import pprint
FPATHS = dict(
    data={
        "filtered":  "Data-NLP/movie_reviews_v2.csv",
        "processed": "Data-NLP/processed_data.joblib",
        "ml": {
            "train": "Data-NLP/training-data.joblib",  
            "test": "Data-NLP/testing-data.joblib",
            
            "label_encoder":"Data-NLP/label-encoder.joblib",
            'target_lookup':"Data-NLP/target-lookup-dict.joblib",
            
        },
        "tf": {
            "train_tf": "Data-NLP/training_tf",  
            "test_tf": "Data-NLP/testing_tf", 
         },           
    },
    models={
        "rf": "models/rf-pipe.joblib", 
        "nb": "models/nb-pipe.joblib", 
        "gru": "models/tf/"
    },
    eda = {
        "scattertext": "eda/scattertext.html",
        "wordclouds-raw": "eda/wordclouds-raw-text.png",
        'wordclouds-lemmas':"eda/wordclouds-lemmas.png",
    }
)
pprint(FPATHS)

{'data': {'filtered': 'Data-NLP/movie_reviews_v2.csv',
          'ml': {'label_encoder': 'Data-NLP/label-encoder.joblib',
                 'target_lookup': 'Data-NLP/target-lookup-dict.joblib',
                 'test': 'Data-NLP/testing-data.joblib',
                 'train': 'Data-NLP/training-data.joblib'},
          'processed': 'Data-NLP/processed_data.joblib',
          'tf': {'test_tf': 'Data-NLP/testing_tf',
                 'train_tf': 'Data-NLP/training_tf'}},
 'eda': {'scattertext': 'eda/scattertext.html',
         'wordclouds-lemmas': 'eda/wordclouds-lemmas.png',
         'wordclouds-raw': 'eda/wordclouds-raw-text.png'},
 'models': {'gru': 'models/tf/',
            'nb': 'models/nb-pipe.joblib',
            'rf': 'models/rf-pipe.joblib'}}


In [19]:
# Save the filepaths in a config folder
import os, json
os.makedirs('config/', exist_ok=True)
FPATHS_FILE = 'config/filepaths.json'
with open(FPATHS_FILE, 'w') as f:
    json.dump(FPATHS, f)

In [20]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import spacy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import tensorflow as tf
# Increase column width
pd.set_option('display.max_colwidth', 250)
import tensorflow as tf

In [21]:
import os, sys
%load_ext autoreload 
%autoreload 2
import movie_functions as fn

In [22]:
# Load Data from your path
df = pd.read_csv("Data-NLP/movie_reviews_v2.csv",
                 index_col = 'movie_id')
len(df)

1989

In [23]:
# Keep only 4=< and 9=>reviews (binary classification)
df = df.loc[ df['rating'].isin([1,2,3,4,9,10])]
# We will split on each space, and then get the length
df['sequence_length'] =df['review'].map( lambda x: len(x.split(" ")))
# Define a filter to identify reviews less than 400 words
filter_short = df['sequence_length']< 400
# Keep short reviews (<400 words)
df=  df.loc[filter_short]
# Drop unnecessary columns
df= df.drop(columns = ['sequence_length']) #'review_id', 'imdb_id'])
df.head()

,original_title,review,rating
movie_id,,,
7443,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0
955,Mission: Impossible II,"The first underwhelmed me, but this one straight-up bored me. Again, of course seeing Hunt climb a mountain without a harness is impressive sure. And I even quite liked the idea behind the villain of the piece (though even that angle was woefully underdeveloped).\r\n\r\nEven setting it in predom...",3.0
955,Mission: Impossible II,After quite entertainign Mission Impossible I the second installment turned out ... terrible. As if the screenwriters didn't know how to fill the 2 hrs with action the overuse of heroic slow motion scenes is horrible. You almost might need a barf bag if you can't stand slo-mo every five seconds....,2.0
2621,Return to Me,"Okay, I will admit I can be a bit of an easy grader when it comes to romantic comedies, as long as they are witty with likable characters, don’t insult my intelligence and have suitable happy endings (I guess if they don’t end happily, they aren’t rom-coms).\r\n\r\nI saw this movie many years ag...",9.0
4234,Scream 3,"**_Scream 3’s_ lackluster screenplay and unimaginative kills leave a film that is a bore to watch.**\r\n\r\nThe meta-narrative of trilogies throughout the film does not make up for how abysmal the plot was. This film creates so much lore for the past movies seemingly out of the blue, muddling up...",4.0


In [24]:
df['rating'].value_counts()

9.0     481
10.0    476
4.0     448
1.0     243
3.0     222
2.0     119
Name: rating, dtype: int64

In [25]:
# We can define the path using our dictionary
fpath = FPATHS['data']['filtered']
df.to_csv(fpath)

In [26]:
# Mapping numeric value with public-facing label
rename_target_map = {9-10:'High_Rating', 1-4:"Low_Rating"}

In [27]:
# Replace y values
y = df['rating'].replace(rename_target_map)
# Define X as normal
X = df['review']

In [28]:
# Train test split
X_train, X_test, y_train_str, y_test_str = train_test_split(X,y,test_size=0.25,
                                                    random_state=321)

In [29]:
# Getting unique classes to convert 
class_names = y_train_str.unique()
class_names

array([ 9.,  3., 10.,  2.,  4.,  1.])

In [30]:
from sklearn.preprocessing import LabelEncoder
# Fit the label encoder on unique class names
encoder = LabelEncoder()
encoder.fit(class_names)

LabelEncoder()

In [31]:
# Transform the y_train_str and y_test_str with encoder
y_train = encoder.transform(y_train_str)
y_test = encoder.transform(y_test_str)
y_train[:10]

array([4, 2, 5, 5, 4, 4, 4, 1, 4, 2], dtype=int64)

In [32]:
# Extract the class names from the encoder
classes = encoder.classes_
classes

array([ 1.,  2.,  3.,  4.,  9., 10.])

In [33]:
# Get the encoded values for each both classes
class_codes = encoder.transform(classes)
class_codes

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [34]:
# Making lookup dictionary to find the encoded label's original name
target_lookup = dict(zip(class_codes,classes ))
target_lookup

{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 9.0, 5: 10.0}

In [35]:
# Check for class balance
pd.Series(y_train).value_counts(normalize=True)

4    0.246144
5    0.240778
3    0.220657
0    0.126090
2    0.108652
1    0.057679
dtype: float64

In [36]:
# Use RUS to balance the classes
sampler = RandomUnderSampler(random_state=42)
X_train, y_train = sampler.fit_resample(X_train.values.reshape(-1,1),y_train)
X_train = X_train.flatten()

# Check for class balance
pd.Series(y_train).value_counts(normalize=True)

0    0.166667
1    0.166667
2    0.166667
3    0.166667
4    0.166667
5    0.166667
dtype: float64

In [37]:
# Saving the training data
fpath_train  = FPATHS['data']['ml']['train']
joblib.dump([X_train, y_train], fpath_train)
# Saving the testing data
fpath_test = FPATHS['data']['ml']['test']
joblib.dump([X_test, y_test], fpath_test)

['Data-NLP/testing-data.joblib']

In [38]:
# Save encoder
fpath_encoder = FPATHS['data']['ml']['label_encoder']
joblib.dump(encoder,fpath_encoder)

# Save dictionary
fpath_lookup = FPATHS['data']['ml']['target_lookup']
joblib.dump(target_lookup,fpath_lookup )

['Data-NLP/target-lookup-dict.joblib']

## Modeling: Random Forest & MultinomialNB

### Random Forest

In [39]:
# Create a pipeline with a vectorizer and classification model.
rf_pipe = Pipeline([('vectorizer', CountVectorizer(stop_words="english")),
                     ('clf',RandomForestClassifier(random_state=42))])
# Fit the model pipeline
rf_pipe.fit(X_train, y_train)
# Evaluate
fn.evaluate_classification(rf_pipe, X_train,y_train, X_test, y_test)

----------------------------------------------------------------------
 Classification Metrics: Training Data
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        86
           1       1.00      1.00      1.00        86
           2       1.00      1.00      1.00        86
           3       1.00      1.00      1.00        86
           4       1.00      1.00      1.00        86
           5       1.00      1.00      1.00        86

    accuracy                           1.00       516
   macro avg       1.00      1.00      1.00       516
weighted avg       1.00      1.00      1.00       516




C:\Users\Valde\OneDrive\#CodingDojo\03-DataEnrichment\Data-Enrichment-Project\movie_functions.py:255: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\Valde\OneDrive\#CodingDojo\03-DataEnrichment\Data-Enrichment-Project\movie_functions.py:255: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


----------------------------------------------------------------------
 Classification Metrics: Test Data
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.28      0.49      0.35        55
           1       0.22      0.21      0.22        33
           2       0.51      0.40      0.45        60
           3       0.76      0.42      0.54       119
           4       0.56      0.35      0.43       114
           5       0.41      0.65      0.50       117

    accuracy                           0.45       498
   macro avg       0.46      0.42      0.42       498
weighted avg       0.51      0.45      0.45       498



In [40]:
fpath_rf_pipe = FPATHS['models']['rf']
joblib.dump(rf_pipe, fpath_rf_pipe)

['models/rf-pipe.joblib']

### Multinomial Naive Bayes:

In [41]:
# Create a pipeline with a vectorizer and classification model.
nb_pipe = Pipeline([('vectorizer', CountVectorizer(stop_words="english")),
                     ('clf',MultinomialNB())])
# clf_nb = MultinomialNB()
nb_pipe.fit(X_train, y_train)
fn.evaluate_classification(nb_pipe, X_train,y_train, X_test, y_test)

----------------------------------------------------------------------
 Classification Metrics: Training Data
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      0.98      0.96        86
           1       0.96      0.94      0.95        86
           2       0.99      0.98      0.98        86
           3       0.98      0.98      0.98        86
           4       0.97      0.98      0.97        86
           5       0.98      0.97      0.97        86

    accuracy                           0.97       516
   macro avg       0.97      0.97      0.97       516
weighted avg       0.97      0.97      0.97       516


----------------------------------------------------------------------
 Classification Metrics: Test Data
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.28      0.62      0.38

C:\Users\Valde\OneDrive\#CodingDojo\03-DataEnrichment\Data-Enrichment-Project\movie_functions.py:255: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\Valde\OneDrive\#CodingDojo\03-DataEnrichment\Data-Enrichment-Project\movie_functions.py:255: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [42]:
fpath_nb_pipe = FPATHS['models']['nb']
joblib.dump(nb_pipe, fpath_nb_pipe)

['models/nb-pipe.joblib']

# Part 2: Streamlit App

### You will create a Streamlit app to get model predictions for user-entered text. 

### You may select `either your best machine learning model or deep nlp model.` **Note:** for portfolio purposes, it would be best to eventually create an app for both.
- #### Create a Stramlit app for getting predictions for a user-entered text from your loaded model.
- #### (Optional but recommended); Include a Lime Text Explainer explanation for the prediction.
- #### Include the ability to load the training and test data to evaluate the model.

In [43]:
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import json
with open('config/filepaths.json') as f:
    FPATHS = json.load(f)
st.title("Predicting Yelp Review Ratings")

DeltaGenerator()

In [44]:
# Get text to predict from the text input box
X_to_pred = st.text_input("### Enter text to predict here:", 
                          value="Great a must See!.")

### Getting Predictions

In [45]:
# Loading the ML model
@st.cache_resource
def load_ml_model(fpath):
    loaded_model = joblib.load(fpath)
    return loaded_model
# Load model from FPATHS
model_fpath = FPATHS['models']['nb']
nb_pipe = load_ml_model(model_fpath)

In [46]:
import joblib
# load target lookup dict
@st.cache_data
def load_lookup(fpath=FPATHS['data']['ml']['target_lookup']):
    return joblib.load(fpath)
@st.cache_resource
def load_encoder(fpath=FPATHS['data']['ml']['label_encoder'] ):
    return joblib.load(fpath)

2024-04-22 02:43:08.971 No runtime found, using MemoryCacheStorageManager


In [47]:
# Load the target lookup dictionary
target_lookup = load_lookup()
target_lookup

2024-04-22 02:43:31.148 No runtime found, using MemoryCacheStorageManager


{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 9.0, 5: 10.0}

In [48]:
# Load the encoder
encoder = load_encoder()
encoder.classes_

array([ 1.,  2.,  3.,  4.,  9., 10.])

In [51]:
# Basic Function to obtain prediction
def make_prediction(X_to_pred):
    pred_class = nb_pipe.predict([X_to_pred])[0]
    return pred_class

In [52]:
# Test function in notebook
pred_class = make_prediction(X_to_pred)
pred_class

5

In [56]:
# Update the function to decode the prediction
def make_prediction(X_to_pred, nb_pipe=nb_pipe, lookup_dict= target_lookup):
    # Get Prediction
    pred_class = nb_pipe.predict([X_to_pred])[0]
    # Decode label
    pred_class = lookup_dict[pred_class]
    return pred_class

In [57]:
# Test updated function
pred_class = make_prediction(X_to_pred)
pred_class

10.0

In [58]:
from lime.lime_text import LimeTextExplainer
@st.cache_resource
def get_explainer(class_names = None):
    lime_explainer = LimeTextExplainer(class_names=class_names)
    return lime_explainer
    
def explain_instance(explainer, X_to_pred, predict_func):
    explanation = explainer.explain_instance(X_to_pred, predict_func)
    return explanation.as_html(predict_proba=False)
# Create the lime explainer
explainer = get_explainer(class_names = encoder.classes_)

In [60]:
# Demonstrating getting class probability predictions from pipeline.predict_proba 
nb_pipe.predict_proba([X_to_pred])

array([[0.09983913, 0.10213241, 0.0999401 , 0.11223375, 0.28645085,
        0.29940376]])

In [62]:
# Obtaining an explanation for our X_to_pred text
explanation = explain_instance(explainer, X_to_pred, predict_func=rf_pipe.predict_proba)
# To display in the notebook
from IPython.display import display, HTML
display(HTML(explanation))

C:\Users\Valde\anaconda3\envs\dojo-env\lib\site-packages\lime\explanation.py:251: ResourceWarning: unclosed file <_io.TextIOWrapper name='C:\\Users\\Valde\\anaconda3\\envs\\dojo-env\\lib\\site-packages\\lime\\bundle.js' mode='r' encoding='utf8'>
  encoding="utf8").read()


### Classification Metrics:

In [63]:
## Loading our training and test data
@st.cache_data
def load_Xy_data(joblib_fpath):
    return joblib.load(joblib_fpath)
# Load training data from FPATHS
train_data_fpath  = FPATHS['data']['ml']['train']
X_train, y_train = load_Xy_data(train_data_fpath)
# Load test data from FPATHS
test_data_fpath  = FPATHS['data']['ml']['test']
X_test, y_test = load_Xy_data(test_data_fpath)

2024-04-22 02:55:19.969 No runtime found, using MemoryCacheStorageManager
2024-04-22 02:55:19.972 No runtime found, using MemoryCacheStorageManager


### Preparing tensorflow Models:

In [65]:
# Open the file structure dictionary
import json
with open('config/filepaths.json') as f:
    FPATHS = json.load(f)
FPATHS

{'data': {'filtered': 'Data-NLP/movie_reviews_v2.csv',
  'processed': 'Data-NLP/processed_data.joblib',
  'ml': {'train': 'Data-NLP/training-data.joblib',
   'test': 'Data-NLP/testing-data.joblib',
   'label_encoder': 'Data-NLP/label-encoder.joblib',
   'target_lookup': 'Data-NLP/target-lookup-dict.joblib'},
  'tf': {'train_tf': 'Data-NLP/training_tf',
   'test_tf': 'Data-NLP/testing_tf'}},
 'models': {'rf': 'models/rf-pipe.joblib',
  'nb': 'models/nb-pipe.joblib',
  'gru': 'models/tf/'},
 'eda': {'scattertext': 'eda/scattertext.html',
  'wordclouds-raw': 'eda/wordclouds-raw-text.png',
  'wordclouds-lemmas': 'eda/wordclouds-lemmas.png'}}

In [66]:
# load training data using dictionary
[X_train, y_train] = joblib.load(FPATHS['data']['ml']['train'])
[X_test, y_test]  = joblib.load(FPATHS['data']['ml']['test'])
X_train[:5],y_train[:5]

(array(["Easily the worst film I have ever seen that wasn't helmed by a female director. I am usually very picky when it comes to selecting movies to watch, hence my usually high satisfaction as a cinephile. I can't believe I wasted my time on this vile piece of garbage and that this ever saw a cinematic release when there MUST be thousands...NO, millions, of better things that could have been made instead. The Lumiere brothers must be rolling in their graves at how cinema has been travestied.\r\n\r\nMy only regret other than my time wasted and the scar on my brain from having watched it is the fact that unlike Metacritic, on themoviedb.org I can't give it a zero rating. Or a negative, for that matter! =)",
        "So Creeds kid as a good job and boxes on the side for reasons... and wins.\r\n\r\nBut he doesn't want to do underground boxing, he wasn't to make it big like his dad, because he's the bestest of the bestest.\r\n\r\nAnd there is a LITTLE struggle here, a little of the heros 

In [67]:
# Make a dataset object from train data
train_tf = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# Make a dataset object from test data
test_tf = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [68]:
# Shuffle dataset
train_tf = train_tf.shuffle(buffer_size=len(train_tf),reshuffle_each_iteration=False)

In [69]:
len(train_tf), len(test_tf)

(516, 498)

In [73]:
# Shuffle dataset
loaded_train =\
loaded_train.shuffle(buffer_size=len(loaded_train),reshuffle_each_iteration=False)

NameError: name 'loaded_train' is not defined

In [74]:
# Set the ratio of the train, validation, test split
split_train = .75
split_val =  .25
# Calculate the number of samples for training and validation data 
n_train_samples =  int(len(train_tf) * split_train)
n_val_samples = int(len(train_tf) * split_val)
# Set the batch size
BATCH_SIZE =32
import math
# math.ceil will round up
# How many batches? 
n_train_batches = math.ceil(n_train_samples/BATCH_SIZE)
n_val_batches = math.ceil(n_val_samples/BATCH_SIZE)
print(f"    - train:\t{n_train_samples} samples \t({n_train_batches} batches)")
print(f"    - val:  \t{n_val_samples} samples \t({n_val_batches} batches)")

    - train:	387 samples 	(13 batches)
    - val:  	129 samples 	(5 batches)


In [75]:
# Use take and skip to define each set
train_ds = train_tf.take(n_train_samples).batch(batch_size=BATCH_SIZE)
# Skip over the training batches and take the validation batches
val_ds = train_tf.skip(n_train_samples).take(n_val_samples).batch(batch_size=BATCH_SIZE)
# Confirm the number of batches in each
print (f' There are {len(train_ds)} training batches.')
print (f' There are {len(val_ds)} validation batches.')

 There are 13 training batches.
 There are 5 validation batches.


In [76]:
# How many samples in the test data
n_test_samples = len(test_tf)
n_test_samples

498

In [77]:
# Put the test data into batches also
test_ds = test_tf.batch(batch_size = BATCH_SIZE)
# How many batches
print (f' There are {len(test_ds)} testing batches.')

 There are 16 testing batches.


In [78]:
# Save training dataset object
train_ds_fpath = FPATHS['data']['tf']['train_tf']
tf.data.Dataset.save(train_ds, train_ds_fpath)
# Save testing dataset object
test_ds_fpath = FPATHS['data']['tf']['test_tf']
tf.data.Dataset.save(test_ds, test_ds_fpath)

In [79]:
# Define classes
# classes = np.unique(example_y)
encoder = joblib.load(FPATHS['data']['ml']['label_encoder'])
classes = encoder.classes_
classes

array([ 1.,  2.,  3.,  4.,  9., 10.])

### Modeling: GRU saved at the end of part 6 Notebook

In [94]:
@st.cache_resource
def load_network(fpath):
    model = tf.keras.models.load_model(fpath)
    return model

fpath_model = FPATHS['models']['gru_model']

best_network = load_network(fpath_model)

KeyError: 'gru_model'

In [95]:
import joblib
# load target lookup dict
@st.cache_data
def load_lookup(fpath=FPATHS['data']['ml']['target_lookup']):
    return joblib.load(fpath)

target_lookup = load_lookup()

2024-04-22 04:24:48.170 No runtime found, using MemoryCacheStorageManager


In [96]:
def predict_decode_deep(X_to_pred, network,lookup_dict,
                       return_index=True):
    
    if isinstance(X_to_pred, str):
        
        X = [X_to_pred]
    else:
        X = X_to_pred
    
    pred_probs = network.predict(X)

    pred_class = fn.convert_y_to_sklearn_classes(pred_probs)
    
    # Decode label
    class_name = lookup_dict[pred_class[0]]

    return class_name

In [99]:
# Test the function in your notebook
pred_class_name = predict_decode_deep(X_to_pred, best_model,target_lookup) # lookup_neural_net)
pred_class_name

NameError: name 'best_model' is not defined

In [100]:
if st.button("Get prediction."):
        pred_class_name = predict_decode_deep(X_to_pred, best_network,target_lookup)
        st.markdown(f"##### Neural Network Predicted category:  {pred_class_name}")
    
else: 
    st.empty()

In [101]:
st.divider()

st.subheader('Evaluate Neural Network')

## To place the 3 checkboxes side-by-side
col1,col2,col3 = st.columns(3)
show_train = col1.checkbox("Show training data.", value=True)
show_test = col2.checkbox("Show test data.", value=True)

# The end of my 6 Is part of my Part 7 notebook the only thing I'm not 100% sure that I was able to save correctly was the deep learning however, I was able to work with Nicole to get the 1st part saved. 7b is my deployment process. My ml_app is my ML Models.
### Later I followed the NLP prep part which I believe might have erased what I did originally but gave me the outcome that was needed. Not completly sure if my app will work but We will see.